# Importing Libraries and Census Data

In [ ]:
#libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime, timedelta
import re
import statsmodels.formula.api as smf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from wordcloud import WordCloud, STOPWORDS
#reading in census population data for estimating market-size-dictated heterogeneous treatment effects
#in theory, smaller market postings should see a bigger boost in interest from job remoteness
census_data = (pd.read_csv('sub-est2021_all.csv', encoding='latin-1')[['NAME', 'POPESTIMATE2021']].
               rename(columns={'NAME': 'Market', 'POPESTIMATE2021': 'Population'}))
#a handful of hamlets, CDPs, and areas not properly addressed by my cleaning need to be manually specified
missing_mkts = ['King of Prussia', 'Washington, DC', 'Reston', 'Carson City', 'Byron Center', 
                'Chantilly', 'Devens', 'Glen Burnie', 'Hopewell Junction', 'Monmouth Junction', 
                'Newtown Square', 'Plainsboro Center', 'South Hooksett', 'Whippany', 'Whitehouse Station', 
                'Melville', 'Westlake, TX', 'Highlands Ranch', 'Patuxent River', 
                'Annapolis Junction', 'Basking Ridge', 'Beale Air Force Base', 'Boise',
                'Iselin', 'Landisville', 'Lutherville-Timonium', 'Prudhoe Bay',
                'Purchase', 'St Louis Park']
missing_pops = [22028, 712816, 61418, 58993, 6653, 
                23820, 2007, 69649, 725, 8911, 
                13726, 2715, 5911, 8863, 3838, 
                17992, 1683, 107017, 12934, 40, 26747, 
                69536, 237446, 17356, 4771, 15814, 
                1416, 5391, 49158]
#an index starting at the end of the census dataframe
idx = [81416 + i for i in range(len(missing_mkts))]
#cities, towns and counties annoyingly have 'city', 'town', or 'county' at the end of each of their names
#only need to know if they're county
census_data['Market'] = census_data['Market'].apply(lambda x: x.rsplit(' ', 1)[0] if 'county' not in x.lower() else x)
missing_mkts = pd.DataFrame({'Market': missing_mkts, 'Population': missing_pops}, index=idx)
census_data = pd.concat([census_data, missing_mkts], axis=0)
#coercing certain census data spellings to what's apparently used on LinkedIn
census_data['Market'] = census_data['Market'].replace({'Nashville-Davidson metropolitan': 'Nashville', 
                                                       'Louisville/Jefferson County metro government (balance)': 'Louisville', 
                                                       'Bronx County': 'Bronx', 
                                                       'Cranberry': 'Cranberry Township', 
                                                       'Minneapolis': 'Minneapolis–Saint Paul', 
                                                       'St. Paul': 'Minneapolis–Saint Paul',
                                                       'Parsippany-Troy Hills': 'Parsippany', 
                                                       'St. Louis': 'St Louis', 
                                                       'St. Petersburg': 'St Petersburg'})
#considering the Twin Cities one market
census_data.loc[census_data['Market'].str.contains('Minneapolis–Saint Paul'), 'Population'] = 425336+307193
#manually fixing some other issues
census_data.loc[census_data['Market'].str.contains('McLean'), 'Population'] = 48566
census_data.loc[census_data['Market'].str.contains('Lexington'), 'Population'] = 321793
census_data.loc[census_data['Market']=='Hershey', 'Population'] = 14941
census_data.loc[census_data['Market'].str.contains('Wilmington'), 'Population'] = 70750
census_data.loc[census_data['Market'].str.contains('Frankfort'), 'Population'] = 28595
census_data.loc[census_data['Market']=='Fulton', 'Population'] = 2049
#though need to see if any Columbus, OH end up in sample. Diff (bigger) pop. (see hybrid matched sample)
census_data.loc[census_data['Market']=='Columbus', 'Population'] = 50569
#there are plenty of duplicates across states; I'll keep the most populous since it's likeliest that's what's in my data
census_data.sort_values(by=['Population'], ascending=False, inplace=True)
census_data.drop_duplicates(subset=['Market'], keep='first', inplace=True)
#I later audit the 20 or so most and least populous markets to make sure there are no glaring mistakes
#e.g. San Diego city being assigned San Diego County's population

# Reading in Job-Posting Data and Some Processing

In [ ]:
# the first 14 files are all Easy Applys but I didn't track at the time, while remainder are a mix I tracked
#I'm going to treat sentinel values 'Not Available' and 'Error' as NAs
dfs = [pd.read_csv('li_postings.csv', na_values=['Not Available', 'Error']).assign(Easy_Apply=1)]
for i in range(2,823):
    if i < 15:
        dfs.append(pd.read_csv(f'li_postings{i}.csv', na_values=['Not Available', 'Error']).assign(Easy_Apply=1))
    else:
        dfs.append(pd.read_csv(f'li_postings{i}.csv', na_values=['Not Available', 'Error']))
data=pd.concat(dfs).reset_index(drop=True)
#it's only worth dropping observations missing time up and/or applicant values, since those are crucial
data = data[(data['Time Up'].notna())&(data['Applicants'].notna())]
#also, I'm for now making the judgement call to exclude jobs posted less than an hour prior to scraping
data = data[(~data['Time Up'].str.contains('second'))&(~data['Time Up'].str.contains('minute'))]
#granularity/duplicate handling
#all the postings
count = data.shape[0]
#number of postings unique at the Job Title, Company, and Location level
unique_count = data[~data.duplicated(subset=['Job_Title', 'Company', 'Location'])].shape[0]
print(f'{unique_count} of {count} rows are postings unique at the job title-company-location level.')

#accounting for some number of postings advertised as remote or hybrid in the job title; a couple hundred or so
def remoteness_from_title(df):
    remote_words = ['Remote', 'Home', 'WFH', 'REMOTE', 'HOME']
    hybrid_words = ['Hybrid', 'HYBRID']
    if (any(word in df['Job_Title'] for word in remote_words)):
        return 'Remote'
    elif (any(word in df['Job_Title'] for word in hybrid_words)):
        return 'Hybrid'
    else:
        return df['Remoteness']
data['Remoteness'] = data.apply(remoteness_from_title, axis=1)
raw = data.copy()
raw['Remoteness'] = raw['Remoteness'].fillna('Not Mentioned')
#another number of postings' work arrangements are only given in the description; I'm finding that those with 
#job descriptions containing the word 'hybrid' are hybrid, but that this is not necessarily the case for remote jobs, 
#as some hybrid jobs are described as 'hybrid remote' or something like 'remote 3 days a week onsite 2'. 
#Therefore, maybe I only mark jobs having 'remote' and not having that kind of language in their job description as remote. 

# def remoteness_from_desc(df):
#     text = df['Job_Title'].lower()
#     on_site_words = ['on-site', 'onsite', 'on site', 'office', 'hybrid']
#     hybrid_filter = (df['Remoteness'].isna())&('hybrid' in text)
#     remote_filter = (df['Remoteness'].isna())&('remote' in text)&(all(word not in text for word in on_site_words))
#     if hybrid_filter:
#         return 'Hybrid'
#     elif remote_filter:
#         return 'Remote'
#     else:
#         return df['Remoteness']
# data['Remoteness'] = data.apply(remoteness_from_desc, axis=1)

#for now, I'm going to treat companies not clearly advertised as remote or hybrid as on-site. 
data['Remoteness'] = data['Remoteness'].fillna('On-site')

#standardizing job title capitalization for duplicate filtering later on
data['Job_Title'] = data['Job_Title'].apply(lambda string: string.title())

#casting Applicant Count to int; removing commas and needing to pitstop at float for some reason
data['Applicants'] = data['Applicants'].astype(str)
data['Applicants'] = data['Applicants'].apply(lambda x: x.replace(',', '') if ',' in x else x)
data['Applicants'] = data['Applicants'].astype(float)
data['Applicants'] = data['Applicants'].astype(int)

#it might also be worth stripping those remote and hybrid words from the job title if
#text features are used in a propensity score model

# More Processing

In [ ]:
#deriving work basis dummies
data['Contract_Role'] = data.apply(lambda row: int('contract' in str(row.Position_Info).lower()), axis=1)
data['Part_Time_Role'] = data.apply(lambda row: int('part-time' in str(row.Position_Info).lower()), axis=1)
data['Full_Time_Role'] = data.apply(lambda row: int('full-time' in str(row.Position_Info).lower()), axis=1)
data['Missing_Work_Basis'] = data['Full_Time_Role']+data['Part_Time_Role']+data['Contract_Role']
data['Missing_Work_Basis'] = data['Missing_Work_Basis'].replace({1: 0, 0: 1})

#deriving seniority level dummies...
data['Internship'] = data.apply(lambda row: int('internship' in str(row.Position_Info).lower()), axis=1)
data['Entry_Level'] = data.apply(lambda row: int('entry level' in str(row.Position_Info).lower()), axis=1)
data['Associate_Level'] = data.apply(lambda row: int('associate' in str(row.Position_Info).lower()), axis=1)
data['Mid_Senior_Level'] = data.apply(lambda row: int('mid-senior level' in str(row.Position_Info).lower()), axis=1)
data['Director_Level'] = data.apply(lambda row: int('director' in str(row.Position_Info).lower()), axis=1)
data['Executive_Level'] = data.apply(lambda row: int('executive' in str(row.Position_Info).lower()), axis=1)
data['Missing_Seniority'] = (data['Internship']+data['Entry_Level']+data['Associate_Level']+data['Mid_Senior_Level']
                             +data['Director_Level']+data['Executive_Level'])
data['Missing_Seniority'] = data['Missing_Seniority'].replace({1: 0, 0: 1})
#...and a single categorical column for ease of analysis later
#about 70 rows have multiple seniorities listed but all are internships, so they're internships
#see via data[~(data.iloc[:, 17:24].sum(axis=1)>1)]
def seniority_puller(df):
    if df['Internship']==1:
        return 'Internship'
    elif df['Entry_Level']==1:
        return 'Entry Level'
    elif df['Associate_Level']==1:
        return 'Associate Level'
    elif df['Mid_Senior_Level']==1:
        return 'Mid-Senior Level'
    elif df['Director_Level']==1:
        return 'Director Level'
    elif df['Executive_Level']==1:
        return 'Executive Level'
    else:
        return 'Seniority Info Not Given'
data['Seniority'] = data.apply(seniority_puller, axis=1)

#deriving salary info
data['Salary_Info'] = data.apply(lambda row: str(row['Position_Info']).split('·')[0] if '$' in str(row['Position_Info']) else 'Not Given', axis=1)
data['Gives_Salary'] = data.apply(lambda row: 1 if str(row['Salary_Info'])!='Not Given' else 0, axis=1)
def salary_cleaner(info, end='low'):
    if info.count('$')==2:
        if end=='low':
            return info.split('-')[0].split('/')[0].strip('$').replace(',', '')
        else:
            return info.split('-')[1].split('/')[0].strip().strip('$').replace(',', '')
    elif info=='Not Given':
        return '0' #though technically N/A
    else:
        return info.split('/')[0].strip('$').replace(',', '')
def salary_basis(info):
    if '/yr' in info.lower():
        return 'Year'
    elif '/hr' in info.lower():
        return 'Hour'
    elif '/month' in info.lower():
        return 'Month'
    else:
        return 'Not Clear'
#the following ensures no salary info is of some form other than a range or single number
if len(np.array(pd.Series(data.apply(lambda row: row.Salary_Info.count('$'), axis=1)).value_counts().index)) == 3:
    data['Pay_Floor'] = data.apply(lambda row: salary_cleaner(row['Salary_Info'], end='low'), axis=1)
    data['Pay_Ceiling'] = data.apply(lambda row: salary_cleaner(row['Salary_Info'], end='high'), axis=1)
    data['Pay_Basis'] = data.apply(lambda row: salary_basis(row['Salary_Info']), axis=1)

data['Pay_Floor'] = data['Pay_Floor'].astype(float)
data['Pay_Ceiling'] = data['Pay_Ceiling'].astype(float)
data['Pay_Floor_Annual'] = data.apply(lambda row: row['Pay_Floor']*40*52.143 if row['Pay_Basis']=='Hour' else row['Pay_Floor'], axis=1)
data['Pay_Ceiling_Annual'] = data.apply(lambda row: row['Pay_Ceiling']*40*52.143 if row['Pay_Basis']=='Hour' else row['Pay_Ceiling'], axis=1)
data['Pay_Info_Annual_Mean'] = (data['Pay_Floor_Annual']+data['Pay_Ceiling_Annual'])/2
#deriving applicant count per day

#converting to timedelta
def to_timedelta(a_str):
    day_seconds = 86400
    quant = int(a_str.split(' ')[0])
    if 'hour' in a_str:
        return timedelta(hours=quant).seconds / day_seconds
    elif 'day' in a_str:
        return timedelta(days=quant).days
    elif 'week' in a_str:
        return timedelta(weeks=quant).days
    elif 'month' in a_str:
        #convert to avg no. of weeks in a month
        quant*=4.33
        return timedelta(weeks=quant).days
data['Days_Up'] = data.apply(lambda row: to_timedelta(row['Time Up']), axis=1)
#casting Applicant Count to int and getting rate
data['Applicants'] = data['Applicants'].astype(str)
data['Applicants'] = data['Applicants'].apply(lambda x: x.replace(',', '') if ',' in x else x)
data['Applicants'] = data['Applicants'].astype(float)
data['Applicants'] = data['Applicants'].astype(int)
data['Apps_per_Day'] = data['Applicants'] / data['Days_Up']

#pulling company size and industry
def employee_counter(string):
    if ('-' in string) | ('+' in string):
        if '·' in string:
            return string.split('·')[0]
        else:
            return string
    else:
        return 'Not Given'

def industry_puller(string):
    if ('-' in string) | ('+' in string):
        if '·' in string:
            return string.split('·')[1]
        else:
            return 'Not Given'
    elif 'Not Available' in string:
        return 'Not Given'
    else:
        return string
data['Company_Size'] = data.apply(lambda row: employee_counter(str(row['Company_Info'])), axis=1)
data['Company_Size'] = data['Company_Size'].apply(lambda x: x.strip())
data['Industry'] = data.apply(lambda row: industry_puller(str(row['Company_Info'])), axis=1)
data['Industry'] = data['Industry'].apply(lambda x: x.strip())

#pulling location info
def city_area_puller(string):
    if 'Washington, DC' in string:
        return 'Washington, DC'
    elif 'District of Columbia' in string:
        return 'Washington, DC'
    elif 'Washington DC-Baltimore Area' in string:
        return 'Washington, DC'
    elif 'Hartford, CT' in string:
        return 'Hartford'
    elif ('Minneapolis' in string) | ('Paul, MN' in string):
        return 'Minneapolis–Saint Paul'
    elif 'Westlake, TX' in string:
        return 'Westlake, TX'
    elif ',' in string:
        return string.split(',')[0]
    elif 'area' in string.lower():
        if 'salt lake' in string.lower():
            return string.replace('Area', '').replace('Metropolitan', '').replace('Bay', '').replace('County', '').replace('Greater', '').strip()
        else:
            return string.replace('Area', '').replace('Metropolitan', '').replace('City', '').replace('County', '').replace('Greater', '').replace('Bay', '').strip() 
    else:
        return 'Not Available'
        
def state_puller(string):
    if ',' in string:
        if string.split(',')[1]!='United States':
            return string.split(',')[1]
        else:
            return 'Not Available'
    elif string == 'United States':
        return 'Not Available'
    else:
        return 'Further Parsing Needed'

data['Market'] = data.apply(lambda row: city_area_puller(row['Location']), axis=1)
data['Market'] = np.where(data['Market'].str.contains('San Francisco'), 'San Francisco', data['Market'])
data['Market'] = np.where(data['Market'].str.contains('New York'), 'New York', data['Market'])
data['Market'] = np.where(data['Market'].str.contains('Los Angeles'), 'Los Angeles', data['Market'])
data['Market'] = np.where(data['Market'].str.contains('Phoenix'), 'Phoenix', data['Market'])
data['State'] = data.apply(lambda row: state_puller(row['Location']), axis=1)
data['State'] = data['State'].apply(lambda x: x.strip())
data['Market'] = data['Market'].apply(lambda x: x.strip())

#pulling job requirements info; I first look for mentions of any degree before then only caring about highest
def degree_finder(string, *keywords):
    if any(word in string.lower() for word in keywords):
        return 1
    else:
        return 0
associate_degree_keywords = ['associate degree', 'associates degree', 'associate\'s degree', 'junior college degree']
bachelor_degree_keywords = ['bachelor', 'bachelors', 'bachelor\'s']
master_degree_keywords = ['master', 'masters', 'master\'s', 'advanced degree', 'graduate degree']
phd_keywords = ['phd', 'p.h.d', 'doctorate', 'doctor of philosophy', 'ph.d', 'dphil', 'professional degree', 'postdoc']
data['Associate_D'] = data.apply(lambda row: degree_finder(str(row['Job_Description']), *associate_degree_keywords), axis=1)
data['Bachelor_D'] = data.apply(lambda row: degree_finder(str(row['Job_Description']), *bachelor_degree_keywords), axis=1)
data['Master_D'] = data.apply(lambda row: degree_finder(str(row['Job_Description']), *master_degree_keywords), axis=1)
data['PhD'] = data.apply(lambda row: degree_finder(str(row['Job_Description']), *phd_keywords), axis=1)
#adding a highest education column
def highest_education(df):
    if df['PhD']==1:
        return 'PhD'
    elif df['Master_D']==1:
        return 'Master\'s'
    elif df['Bachelor_D']==1:
        return 'Bachelor\'s'
    elif df['Associate_D']==1:
        return 'Associate\'s'
    else:
        return 'No Higher Education Mentioned'
data['Highest_Education'] = data.apply(highest_education, axis=1)
#dropping those dummies
data = data.drop(columns=['Associate_D', 'Bachelor_D', 'Master_D', 'PhD'])
#adding new dummies
data = pd.concat([data, pd.get_dummies(data['Highest_Education'])], axis=1)


#benefits columns
data['four0onek'] = data.apply(lambda row: 1 if '401' in str(row['Benefits']).lower() else 0, axis=1)
data['medical_insurance'] = data.apply(lambda row: 1 if 'medical' in str(row['Benefits']).lower() else 0, axis=1)
data['vision_insurance'] = data.apply(lambda row: 1 if 'vision' in str(row['Benefits']).lower() else 0, axis=1)
data['dental_insurance'] = data.apply(lambda row: 1 if 'dental' in str(row['Benefits']).lower() else 0, axis=1)
data['disability_insurance'] = data.apply(lambda row: 1 if 'disability' in str(row['Benefits']).lower() else 0, axis=1)
data['paid_maternity_leave'] = data.apply(lambda row: 1 if 'paid maternity' in str(row['Benefits']).lower() else 0, axis=1)
data['paid_paternity_leave'] = data.apply(lambda row: 1 if 'paid_paternity' in str(row['Benefits']).lower() else 0, axis=1)
data['tuition_assistance'] = data.apply(lambda row: 1 if 'tuition' in str(row['Benefits']).lower() else 0, axis=1)
data['commuter_benefits'] = data.apply(lambda row: 1 if 'commuter' in str(row['Benefits']).lower() else 0, axis=1)
data['pension'] = data.apply(lambda row: 1 if 'pension' in str(row['Benefits']).lower() else 0, axis=1)

#################################################ongoing efforts to parse out years of experience###################

#removing 'years of age' and 'years old' and turning text to digits to more easily identify years experience requirements
#lowercase text to be turned into digits
lc_numbers = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 
'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen']
#uppercase text to be turned into digits
uc_numbers = [number.title() for number in lc_numbers]
lc_numbers = np.array(lc_numbers)
uc_numbers = np.array(uc_numbers)
#dictionaries to refer to for text-to-digit substitution
lc_digit_dict = {n: i for i, n in zip(np.arange(18).astype(str), lc_numbers)}
uc_digit_dict = {n: i for i, n in zip(np.arange(18).astype(str), uc_numbers)}

#this function will remove references to age in job descriptions and change text numbers to number numbers
def year_digit_processor(string):
    string = re.sub("years of age", '', string)
    string = re.sub("years old", '', string)
    
    #a boolean array indicating what if any text numbers were found
    lc_boolean = np.array(list(number in string for number in lc_numbers))
    if any(lc_boolean): #iterate through each and replace them
        for text in lc_numbers[lc_boolean]:
            string = re.sub(text, lc_digit_dict[text], string)
    else:
        pass
    #same thing for uppercase
    uc_boolean = np.array(list(number in string for number in uc_numbers))
    if any(uc_boolean):
        for text in uc_numbers[uc_boolean]:
            string = re.sub(text, uc_digit_dict[text], string)
    else:
        pass

    return string
data['Job_Description'] = data['Job_Description'].fillna('None Given')
data['Job_Description'] = data.apply(lambda row: year_digit_processor(row['Job_Description']), axis=1)

#regular expressions for identifying years of experience requirements
patterns_pt_1 = r'(\D[0-9] or more year)|(\D[0-9] plus year)|(\D[0-9]-plus year)|(\D[0-9] year)|(\D[0-9][+] year)|'
patterns_pt_2 = r'(1[0-7][+] year)|(1[0-7] or more years)|(1[0-7] plus years)|(1[0-7]-plus years)|(1[0-7] years)|'
patterns_pt_3 = r'([0-9][.]5 year)|(\D[0-9]-year)|(\D[0-9]\) year)|(\D1[0-7]\) year)|(inimum [0-9])|(\D[0-9] [+] year)|'
patterns_pt_4 = r'(\D1[0-7] [+] year)|(inimum [0-9]-[0-9])|(inimum [0-9]-1[0-9])|(inimum 1[0-9]-1[0-9]|)'
patterns_pt_5 = r'(inimum [0-9] to [0-9])|(inimum [0-9] to 1[0-9])|(inimum 1[0-9] to 1[0-9])'
yr_patterns = patterns_pt_1+patterns_pt_2+patterns_pt_3+patterns_pt_4+patterns_pt_5

#a function that will take in a string following any of the above patterns, remove nuisance characters, and return a number
def year_cleaner(string):
    string = string.lower()
    if 'inimum' not in string:#if it wasn't a 'minimum' pattern, first replace all nuisance chracters with spaces
        string = re.sub("[~、\•<>≥()+-,-*'/t:\n–]", ' ', string).strip().strip('-').strip('[').strip(']')
        #then isolate the actual number, dropping any remaining nuisance characters
        fig = string.split(' ')[0]
        fig = re.sub("[A-Za-z>()+-,*'/t]", '', fig)
        #turn to a float
        return float(fig)
    else: #if it was a minimum pattern, strip that part of the string and convert to a float. 
        return float(string.strip('inimum').strip())
        
#this function is what's utilizing the above patterns. 
def reqd_exper_parser(patterns, string):
    #creating a list with entries for every pattern identified
    finds = re.findall(patterns, string)
    try: #there are some intractable formats to be accounted for later on
        if len(finds)!=0: #if any patterns were identified
            #flatten the list. It'll be mostly empty because unidentified patterns are indicated by empty string
            yrs_of_exp = [list(x) for x in finds]
            yrs_of_exp = [elem for a_list in yrs_of_exp for elem in a_list]
            #clean non-empty entries to compute the maximum years requirement listed
            #'if len(year)!=0 is to skip the empty entries'
            yrs_of_exp = np.array([year_cleaner(year.strip('‐')) for year in yrs_of_exp if len(year)!=0], dtype=float)
            return np.max(yrs_of_exp)
        else:
            return 0
    except:
        return 'Something went wrong'
data['Required_Years_Experience'] = data.apply(lambda row: reqd_exper_parser(yr_patterns, row['Job_Description']), axis=1)
#################################################removing duplicates###################

#de-duplicating to uniqueness at the job title, company, remoteness, and location level
#first sorting by those columns and days up in ascending order
#then removing duplicates based on which were scraped first
uniqueness_identifiers = ['Job_Title', 'Company', 'Remoteness', 'Location']
data.sort_values(by=uniqueness_identifiers+['Days_Up'], ascending=True, inplace=True)
data = data.drop_duplicates(subset=uniqueness_identifiers, keep='last')

#disregarding issue for now and casting as numeric
data = data[data['Required_Years_Experience']!='Something went wrong']
data['Required_Years_Experience'] = data['Required_Years_Experience'].astype(float)

#outcome variable
data['Apps_per_Day_log'] = np.log(data['Apps_per_Day']+1)

# Remote/On-Site Matching Sans Job Title

In [ ]:
#filtering to only on-site and remote postings with information on market, industry, and company size
df_rm_os = data.copy()[(data['Remoteness']!='Hybrid')]
df_rm_os = df_rm_os[(df_rm_os['Market']!='Not Available')&(df_rm_os['Industry']!='Not Given')]
#treatment indicator
df_rm_os['Remote'] = np.where(df_rm_os['Remoteness']=='Remote', 1, 0)
#binning posting age, informed by the above visualization
df_rm_os['Posting_Age_Bin'] = pd.cut(df_rm_os['Days_Up'], [0, 1.0, 212.0])
df_rm_os['Posting_Age_Bin'] = df_rm_os['Posting_Age_Bin'].astype(str)
#binning mean salary infos -- most are 0
df_rm_os['Pay_Info_Annual_Mean'] = (df_rm_os['Pay_Floor_Annual']+df_rm_os['Pay_Ceiling_Annual'])/2
df_rm_os['Pay_Info_Mean_Category'] = (pd.cut(df_rm_os['Pay_Info_Annual_Mean'], 
                                               bins=[i*10000 for i in range(26)], 
                                               include_lowest=True)).astype(str)
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Info_Annual_Mean']>250000, 'Over $250,000', df_rm_os['Pay_Info_Mean_Category'])
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Ceiling_Annual']==0, 0, df_rm_os['Pay_Info_Mean_Category'])
#binning years experience requirement info
df_rm_os['yrs_exp_bin'] = pd.cut(df_rm_os['Required_Years_Experience'], 
                                   bins=[i*3 for i in range(4)], 
                                   include_lowest=True)
df_rm_os['yrs_exp_bin'] = np.where(df_rm_os['Required_Years_Experience']>=10, '10 or more', df_rm_os['yrs_exp_bin'])
#creating a list of 6 matching variables
matching_variables1 = ['Market', 'Industry', 'Company_Size', 'Pay_Info_Mean_Category', 'yrs_exp_bin', 'Posting_Age_Bin']
#grouping by that list AND treatment status, then ID'ing duplicates based on matching variables, we can 
#detect where in the covariate space potential matches are present (before gauging text similarity)
matchspots1 = (df_rm_os.groupby(matching_variables1 + ['Remote'])['Job_Title'].
              count().reset_index()).rename(columns={'Job_Title': 'Observations'})
areas = matchspots1.copy()
matchspots1 = (matchspots1[matchspots1.duplicated(subset=matching_variables1)]
              .drop(columns=['Remote', 'Observations']))
matches = [] #to populate with matches
match_id = 0 #to keep track of what matched with what
for matchspot in range(len(matchspots1)): #i.e. for each row of promising covariate values
    #filter the data to observations in that covariate space via a one-to-many inner join
    df = matchspots1.iloc[matchspot, :].to_frame().T
    df = df_rm_os.join(df.set_index(matching_variables1), 
                       on=matching_variables1, 
                       how='inner', 
                       lsuffix='_l', 
                       rsuffix='_r').reset_index(drop=True)
    #I'll cycle through each treated unit and give it a control unit
    treated_df = df.query('Remote==1').reset_index(drop=True)
    control_df = df.query('Remote==0').reset_index(drop=True)
    poss_matches = np.min([len(control_df), len(treated_df)])
    for i in range(poss_matches):
        matches.append(treated_df.iloc[i, :].to_frame().T.assign(Match_Id=match_id))
        matches.append(control_df.iloc[i, :].to_frame().T.assign(Match_Id=match_id))
        match_id += 1 #increment the match identifier
matches = pd.concat(matches)

In [ ]:
matches['Apps_per_Day_log'] = matches['Apps_per_Day_log'].astype(float)
matches['Remote'] = matches['Remote'].astype(int)
matches['Promoted'] = matches['Promoted'].astype(int)
matches['Easy_Apply'] = matches['Easy_Apply'].astype(int)
smf.ols('Apps_per_Day_log~Remote+Promoted+Easy_Apply', matches).fit().summary()

# Remote/On-Site Matching W/ Job Title: tf-idf

In [ ]:
#a quick and dirty way of ensuring references to work arrangement in job titles (e.g. 'remote opportunity')
#aren't used in matching
def title_cleaner(string):
    return (string.replace('Remote', '').replace('Home', '').replace('WFH', '').replace('REMOTE', '')
            .replace('HOME', '').replace('Hybrid', '').replace('HYBRID', '')
            .replace('remote', '').replace('hybrid', '').replace('wfh', '').replace('(', '').replace(')', '').strip())

In [ ]:
#filtering to only on-site and remote postings with information on market, industry, and company size
df_rm_os = data.copy()[(data['Remoteness']!='Hybrid')]
df_rm_os = df_rm_os[(df_rm_os['Market']!='Not Available')&(df_rm_os['Industry']!='Not Given')]
#treatment indicator
df_rm_os['Remote'] = np.where(df_rm_os['Remoteness']=='Remote', 1, 0)
#binning posting age, informed by the above visualization
df_rm_os['Posting_Age_Bin'] = pd.cut(df_rm_os['Days_Up'], [0, 1.0, 212.0])
df_rm_os['Posting_Age_Bin'] = df_rm_os['Posting_Age_Bin'].astype(str)
#binning mean salary infos -- most are 0
df_rm_os['Pay_Info_Annual_Mean'] = (df_rm_os['Pay_Floor_Annual']+df_rm_os['Pay_Ceiling_Annual'])/2
df_rm_os['Pay_Info_Mean_Category'] = (pd.cut(df_rm_os['Pay_Info_Annual_Mean'], 
                                               bins=[i*10000 for i in range(26)], 
                                               include_lowest=True)).astype(str)
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Info_Annual_Mean']>250000, 'Over $250,000', df_rm_os['Pay_Info_Mean_Category'])
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Ceiling_Annual']==0, 0, df_rm_os['Pay_Info_Mean_Category'])
#binning years experience requirement info
df_rm_os['yrs_exp_bin'] = pd.cut(df_rm_os['Required_Years_Experience'], 
                                   bins=[i*3 for i in range(4)], 
                                   include_lowest=True)
df_rm_os['yrs_exp_bin'] = np.where(df_rm_os['Required_Years_Experience']>=10, '10 or more', df_rm_os['yrs_exp_bin'])
#creating a list of 6 matching variables
matching_variables1 = ['Market', 'Industry', 'Company_Size', 'Pay_Info_Mean_Category', 'yrs_exp_bin', 'Posting_Age_Bin']
#grouping by that list AND treatment status, then ID'ing duplicates based on matching variables, we can 
#detect where in the covariate space potential matches are present (before gauging text similarity)
matchspots1 = (df_rm_os.groupby(matching_variables1 + ['Remote'])['Job_Title'].
              count().reset_index()).rename(columns={'Job_Title': 'Observations'})
matchspots1 = (matchspots1[matchspots1.duplicated(subset=matching_variables1)]
              .drop(columns=['Remote', 'Observations']))

#Now finally I'll trawl through those covariate spaces and log t/c pairings similar enough on title as matches
matches = [] #to populate with matches
match_id = 0 #to keep track of what matched with what

for matchspot in range(len(matchspots1)): #i.e. for each row of promising covariate values
    #filter the data to observations in that covariate space via a one-to-many inner join
    df = matchspots1.iloc[matchspot, :].to_frame().T
    df = df_rm_os.join(df.set_index(matching_variables1), 
                       on=matching_variables1, 
                       how='inner', 
                       lsuffix='_l', 
                       rsuffix='_r').reset_index(drop=True)
    #record a cutoff pt. between existing and forthcoming columns; only the latter will be used to gauge similarity
    col_count = df.shape[1]
    #a vector of titles, stripped of any references to treatment with the title_cleaner function
    titles = [title_cleaner(title) for title in df['Job_Title'].copy()]
    #initializing a model that records all unigrams as features,  fitting it on titles, and cleaning future col names
    vect = TfidfVectorizer(ngram_range=(1,1))
    vect.fit(titles)
    feats = [feat.replace(' ', '') for feat in vect.get_feature_names()]
    #appending each posting's new features; use of Jaccard distance to gauge similarity necessitates Boolean
    titles_matrix = vect.transform(titles)
    text_df = pd.DataFrame(titles_matrix.toarray().astype(bool), columns=feats).reset_index(drop=True)
    df = pd.concat([df, text_df], axis=1)
    #features in hand, I'll cycle through each treated unit in the covariate space and find
    #the control neighbor nearest to it based on those features (conditional on exact-matching legwork already done)
    treated_df = df.query('Remote==1').reset_index(drop=True)
    control_df = df.query('Remote==0').reset_index(drop=True)
    #the model is re-fit for every treated unit in case I want to match without replacement,
    #though the order-dependence of that might really hurt/destabilize the matching process
    for i in range(len(treated_df)):
        #a control unit model; predictions with it surface control neighbors
        #(outcome variable Apps per Day not used)
        control_neighbor_finder = (NearestNeighbors(n_neighbors=1).
                                   fit(np.array(control_df.iloc[:, col_count:]), control_df['Apps_per_Day'])
                                  )
        #the treated unit's text features as boolean values
        tdatum = np.array(treated_df.iloc[i, col_count:].to_frame().T, dtype=bool)
        #the treated unit's distance from its nearest neighbor and the NN itself's index value in control_df
        dist, neigh_loc = control_neighbor_finder.kneighbors(tdatum, n_neighbors=1)
        dist, neigh_loc = dist[0][0], neigh_loc[0][0] #extracting scalar values
        #if it's pretty close in job title to a control title in the covariate space, record both
        if dist < 2: #this is subjective. But I think strict enough using Jaccard distance
            matches.append(treated_df.iloc[i, :col_count].to_frame().T.assign(Title_Match_Id=match_id))
            matches.append(control_df.iloc[neigh_loc, :col_count].to_frame().T.assign(Title_Match_Id=match_id))
            match_id += 1 #increment the match identifier
        else:
            pass #since the treated unit's possible match was a poor one, discard it
#turn the list of matched observations into a dataframe
matches1 = pd.concat(matches)
t1 = matches1.query("Remote==1")
c1 = matches1.query("Remote==0")
print('The sample size is now', len(matches1))
print(f'However, this is composed of {len(t1)} treated units')
unique_c_count = c1[~c1.duplicated(subset=['Job_Title', 'Time Up', 'Date_Scraped'])].shape[0]
print(f'and {unique_c_count} control units.')
#re-casting outcome / treatment variables
matches1['Apps_per_Day_log'] = matches1['Apps_per_Day_log'].astype(float)
matches1['Remote'] = matches1['Remote'].astype(int)
matches1['Promoted'] = matches1['Promoted'].astype(int)
matches1['Easy_Apply'] = matches1['Easy_Apply'].astype(int)
smf.ols('Apps_per_Day_log~Remote+Easy_Apply+Promoted', matches1).fit().summary()

# Remote/On-Site Matching W/ Job Title: Jaccard Distance

In [ ]:
#filtering to only on-site and remote postings with information on market, industry, and company size
df_rm_os = data.copy()[(data['Remoteness']!='Hybrid')]
df_rm_os = df_rm_os[(df_rm_os['Market']!='Not Available')&(df_rm_os['Industry']!='Not Given')]
#treatment indicator
df_rm_os['Remote'] = np.where(df_rm_os['Remoteness']=='Remote', 1, 0)
#binning posting age, informed by the above visualization
df_rm_os['Posting_Age_Bin'] = pd.cut(df_rm_os['Days_Up'], [0, 1.0, 212.0])
df_rm_os['Posting_Age_Bin'] = df_rm_os['Posting_Age_Bin'].astype(str)
#binning mean salary infos -- most are 0
df_rm_os['Pay_Info_Annual_Mean'] = (df_rm_os['Pay_Floor_Annual']+df_rm_os['Pay_Ceiling_Annual'])/2
df_rm_os['Pay_Info_Mean_Category'] = (pd.cut(df_rm_os['Pay_Info_Annual_Mean'], 
                                               bins=[i*10000 for i in range(26)], 
                                               include_lowest=True)).astype(str)
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Info_Annual_Mean']>250000, 'Over $250,000', df_rm_os['Pay_Info_Mean_Category'])
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Ceiling_Annual']==0, 0, df_rm_os['Pay_Info_Mean_Category'])
#binning years experience requirement info
df_rm_os['yrs_exp_bin'] = pd.cut(df_rm_os['Required_Years_Experience'], 
                                   bins=[i*3 for i in range(4)], 
                                   include_lowest=True)
df_rm_os['yrs_exp_bin'] = np.where(df_rm_os['Required_Years_Experience']>=10, '10 or more', df_rm_os['yrs_exp_bin'])
#creating a list of 6 matching variables
matching_variables1 = ['Market', 'Industry', 'Company_Size', 'Pay_Info_Mean_Category', 'yrs_exp_bin', 'Posting_Age_Bin']
#grouping by that list AND treatment status, then ID'ing duplicates based on matching variables, we can 
#detect where in the covariate space potential matches are present (before gauging text similarity)
matchspots1 = (df_rm_os.groupby(matching_variables1 + ['Remote'])['Job_Title'].
              count().reset_index()).rename(columns={'Job_Title': 'Observations'})
matchspots1 = (matchspots1[matchspots1.duplicated(subset=matching_variables1)]
              .drop(columns=['Remote', 'Observations']))

#Now finally I'll trawl through those covariate spaces and log t/c pairings similar enough on title as matches
matches = [] #to populate with matches
match_id = 0 #to keep track of what matched with what

for matchspot in range(len(matchspots1)): #i.e. for each row of promising covariate values
    #filter the data to observations in that covariate space via a one-to-many inner join
    df = matchspots1.iloc[matchspot, :].to_frame().T
    df = df_rm_os.join(df.set_index(matching_variables1), 
                       on=matching_variables1, 
                       how='inner', 
                       lsuffix='_l', 
                       rsuffix='_r').reset_index(drop=True)
    #record a cutoff pt. between existing and forthcoming columns; only the latter will be used to gauge similarity
    col_count = df.shape[1]
    #a vector of titles, stripped of any references to treatment with the title_cleaner function
    titles = [title_cleaner(title) for title in df['Job_Title'].copy()]
    #initializing a model that records all unigrams as features,  fitting it on titles, and cleaning future col names
    vect = CountVectorizer(ngram_range=(1,1))
    vect.fit(titles)
    feats = [feat.replace(' ', '') for feat in vect.get_feature_names()]
    #appending each posting's new features; use of Jaccard distance to gauge similarity necessitates Boolean
    titles_matrix = vect.transform(titles)
    text_df = pd.DataFrame(titles_matrix.toarray().astype(bool), columns=feats).reset_index(drop=True)
    df = pd.concat([df, text_df], axis=1)
    #features in hand, I'll cycle through each treated unit in the covariate space and find
    #the control neighbor nearest to it based on those features (conditional on exact-matching legwork already done)
    treated_df = df.query('Remote==1').reset_index(drop=True)
    control_df = df.query('Remote==0').reset_index(drop=True)
    #the model is re-fit for every treated unit in case I want to match without replacement,
    #though the order-dependence of that might really hurt/destabilize the matching process
    for i in range(len(treated_df)):
        #a control unit model; predictions with it surface control neighbors
        #(outcome variable Apps per Day not used)
        control_neighbor_finder = (NearestNeighbors(n_neighbors=1, metric='jaccard').
                                   fit(np.array(control_df.iloc[:, col_count:]), control_df['Apps_per_Day'])
                                  )
        #the treated unit's text features as boolean values
        tdatum = np.array(treated_df.iloc[i, col_count:].to_frame().T, dtype=bool)
        #the treated unit's distance from its nearest neighbor and the NN itself's index value in control_df
        dist, neigh_loc = control_neighbor_finder.kneighbors(tdatum, n_neighbors=1)
        dist, neigh_loc = dist[0][0], neigh_loc[0][0] #extracting scalar values
        #if it's pretty close in job title to a control title in the covariate space, record both
        if dist < .6: #this is subjective. But I think strict enough using Jaccard distance
            matches.append(treated_df.iloc[i, :col_count].to_frame().T.assign(Title_Match_Id=match_id))
            matches.append(control_df.iloc[neigh_loc, :col_count].to_frame().T.assign(Title_Match_Id=match_id))
            match_id += 1 #increment the match identifier
        else:
            pass #since the treated unit's possible match was a poor one, discard it
#turn the list of matched observations into a dataframe
matches2 = pd.concat(matches)
#finally, matching completed, it's maybe worth noting how small the unique set of control observations is
#if way smaller, we're relying on a relatively small set of control units, increasing variance
#a way to offset this is to increase the distance threshold (e.g. from .6 to .7), which of course will increase bias
#another way is to match without replacement, which also increases bias because this also worsens average match quality
t1 = matches2.query("Remote==1")
c1 = matches2.query("Remote==0")
print('The sample size is now', len(matches2))
print(f'However, this is composed of {len(t1)} treated units')
unique_c_count = c1[~c1.duplicated(subset=['Job_Title', 'Time Up', 'Date_Scraped'])].shape[0]
print(f'and {unique_c_count} control units.')

#### Some More Processing+Estimation

In [ ]:
#joining market sizes according to Census data
matches2 = matches2.merge(census_data, on='Market', how='left')
matches2['Population_thousands'] = matches2['Population']/1000
matches2['Population_hundred_thousands'] = matches2['Population']/100000
matches2['Pop_under_1_mil'] = np.where(matches2['Population']<1e6, 1, 0)
matches2['Pay_Floor_Annual'] = matches2['Pay_Floor_Annual'].astype(float)

#re-casting outcome / treatment variables
matches2['Apps_per_Day_log'] = matches2['Apps_per_Day_log'].astype(float)
matches2['Remote'] = matches2['Remote'].astype(int)
matches2['Promoted'] = matches2['Promoted'].astype(int)
matches2['Easy_Apply'] = matches2['Easy_Apply'].astype(int)
matches2_copy = matches2.copy()
matches2_copy['Days_Up'] = matches2_copy['Days_Up'].astype(float)
matches2_copy['Software_Engineer'] = np.where(matches2_copy['Job_Title'].str.contains('Software'), 1, 0)
matches2_copy['Analyst'] = np.where(matches2_copy['Job_Title'].str.contains('Analyst'), 1, 0)
matches2_copy['Associate'] = np.where(matches2_copy['Job_Title'].str.contains('Associate'), 1, 0)
matches2_copy['Senior'] = np.where(matches2_copy['Job_Title'].str.contains('Senior'), 1, 0)
co_size_df = pd.get_dummies(matches2_copy['Company_Size']).drop(columns='10,001+ employees')
co_size_df.columns = ['UnderFiveKEmployees', 'UnderFiveHEmployees', 
                      'UnderTenKEmployees', 'UnderOneKEmployees', 'UnderTwoHEmployees']
co_sizes = '+'.join(list(co_size_df.columns))
cos_todummy = list(matches2_copy['Company'].value_counts()[:20].index)
cos_df = pd.get_dummies(matches2_copy['Company'])[cos_todummy]
cos_df.columns = [re.sub("[,.-/'3&]", '', col).replace(' ', '') for col in cos_df.columns]
cos = '+'.join(list(cos_df.columns))
mkts_todummy = list(matches2_copy['Market'].value_counts()[:20].index)
mkts_df = pd.get_dummies(matches2_copy['Market'])[mkts_todummy]
mkts_df.columns = [re.sub("[,.-/']", '', col).replace(' ', '') for col in mkts_df.columns]
mkts = '+'.join(list(mkts_df.columns))
matches2_copy = pd.concat([matches2_copy, co_size_df, cos_df, mkts_df], axis=1)
smf.ols('Apps_per_Day_log~Remote*Population_hundred_thousands+Easy_Apply+Promoted', matches2_copy).fit().summary()

#### Market Size HTE

In [ ]:
#filtering to only on-site and remote postings with information on market, industry, and company size
df_rm_os = data.copy()[(data['Remoteness']!='Hybrid')]
df_rm_os = df_rm_os[(df_rm_os['Market']!='Not Available')&(df_rm_os['Industry']!='Not Given')]
#treatment indicator
df_rm_os['Remote'] = np.where(df_rm_os['Remoteness']=='Remote', 1, 0)
#binning posting age, informed by the above visualization
df_rm_os['Posting_Age_Bin'] = pd.cut(df_rm_os['Days_Up'], [0, 1.0, 212.0])
df_rm_os['Posting_Age_Bin'] = df_rm_os['Posting_Age_Bin'].astype(str)
#binning mean salary infos -- most are 0
df_rm_os['Pay_Info_Annual_Mean'] = (df_rm_os['Pay_Floor_Annual']+df_rm_os['Pay_Ceiling_Annual'])/2
df_rm_os['Pay_Info_Mean_Category'] = (pd.cut(df_rm_os['Pay_Info_Annual_Mean'], 
                                               bins=[i*10000 for i in range(26)], 
                                               include_lowest=True)).astype(str)
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Info_Annual_Mean']>250000, 'Over $250,000', df_rm_os['Pay_Info_Mean_Category'])
df_rm_os['Pay_Info_Mean_Category'] = np.where(df_rm_os['Pay_Ceiling_Annual']==0, 0, df_rm_os['Pay_Info_Mean_Category'])
#binning years experience requirement info
df_rm_os['yrs_exp_bin'] = pd.cut(df_rm_os['Required_Years_Experience'], 
                                   bins=[i*3 for i in range(4)], 
                                   include_lowest=True)
df_rm_os['yrs_exp_bin'] = np.where(df_rm_os['Required_Years_Experience']>=10, '10 or more', df_rm_os['yrs_exp_bin'])
#creating a list of 6 matching variables
matching_variables1 = ['Market', 'Industry', 'Company_Size', 'Pay_Info_Mean_Category', 'yrs_exp_bin', 'Posting_Age_Bin']
#grouping by that list AND treatment status, then ID'ing duplicates based on matching variables, we can 
#detect where in the covariate space potential matches are present (before gauging text similarity)
matchspots1 = (df_rm_os.groupby(matching_variables1 + ['Remote'])['Job_Title'].
              count().reset_index()).rename(columns={'Job_Title': 'Observations'})
matchspots1 = (matchspots1[matchspots1.duplicated(subset=matching_variables1)]
              .drop(columns=['Remote', 'Observations']))
matches = [] #to populate with matches
match_id = 0 #to keep track of what matched with what
for matchspot in range(len(matchspots1)): #i.e. for each row of promising covariate values
    #filter the data to observations in that covariate space via a one-to-many inner join
    df = matchspots1.iloc[matchspot, :].to_frame().T
    df = df_rm_os.join(df.set_index(matching_variables1), 
                       on=matching_variables1, 
                       how='inner', 
                       lsuffix='_l', 
                       rsuffix='_r').reset_index(drop=True)
    #I'll cycle through each treated unit and give it a control unit
    treated_df = df.query('Remote==1').reset_index(drop=True)
    control_df = df.query('Remote==0').reset_index(drop=True)
    poss_matches = np.min([len(control_df), len(treated_df)])
    for i in range(poss_matches):
        matches.append(treated_df.iloc[i, :].to_frame().T.assign(Match_Id=match_id))
        matches.append(control_df.iloc[i, :].to_frame().T.assign(Match_Id=match_id))
        match_id += 1 #increment the match identifier
matches = pd.concat(matches)

In [ ]:
#casting to float
matches['Apps_per_Day_log'] = matches['Apps_per_Day_log'].astype(float)
matches['Days_Up'] = matches['Days_Up'].astype(float)
#casting to numeric
matches['Remote'] = matches['Remote'].astype(int)
matches['Promoted'] = matches['Promoted'].astype(int)
matches['Easy_Apply'] = matches['Easy_Apply'].astype(int)
#software engineer column
matches['Software_Engineer'] = np.where(matches['Job_Title'].str.contains('Software Engineer'), 1, 0)
#merging in population data; filtering out problematic ones for now
incorrectmkts = ['California', 'Texas', 'Florida', 'Illinois', 'Ohio', 'Georgia', 'Michigan', 'Virginia',
                 'Washington', 'Indiana', 'Missouri', 'Minnesota', 'Kentucky', 'Delaware']
matches = matches[~matches['Market'].isin(incorrectmkts)]
matches = matches.merge(census_data, on='Market', how='left')
matches = matches[matches['Population'].notna()]
# matches['Population_hundred_thousands'] = matches['Population']/100000
#making dummies - company sizes + renaming and joining for model specification
#first filtering to common company sizes
matches = matches[matches['Company_Size'].isin(list(matches['Company_Size'].value_counts().index)[:4])]
co_size_df = pd.get_dummies(matches['Company_Size']).drop(columns='10,001+ employees')
co_size_df.columns = ['UnderFiveKEmployees', 'UnderTenKEmployees', 'UnderOneKEmployees']
co_sizes = '+'.join(list(co_size_df.columns))
#making dummies - companies with at least 50 observations + renaming and joining for model specification
cos_todummy = list(matches['Company'].value_counts()[matches['Company'].value_counts()>=50].index)
cos_df = pd.get_dummies(matches['Company'])[cos_todummy]
cos_df.columns = [re.sub("[,.-/'3&]", '', col).replace(' ', '') for col in cos_df.columns]
cos = '+'.join(list(cos_df.columns))
#making dummies - markets with at least 50 observations + renaming and joining for model specification
mkts_todummy = list(matches['Market'].value_counts()[matches['Market'].value_counts()>=50].index)
#joining dfs and string
matches = pd.concat([matches, co_size_df, cos_df], axis=1)
#randomly sampling 10 markets and ordering them by population for plotting
np.random.seed(2023)
idx = list(np.random.choice(np.arange(len(mkts_todummy)), size=20, replace=False))
mkts20 = [mkts_todummy[i] for i in idx]
mkts20 = list(matches[matches['Market'].isin(mkts20)].groupby('Market')['Population'].mean().sort_values(ascending=False).index)
#extracting HTE estimates
Coefficients = []
Lower_Bounds = []
Upper_Bounds = []
Error_Bars = []
vs2 = co_sizes+'+'+cos
#extract HTE estimates 
for mkt in mkts20:
    df = matches.copy()
    #removing stuff that causes issues in model specification
    label = re.sub("[,.-/–']", '', mkt).replace(' ', '')
    df[f'{label}'] = np.where(df['Market']==mkt, 1, 0)
    vs = f'Apps_per_Day_log~Remote*{label}+Easy_Apply+Promoted+Days_Up+Software_Engineer+'+vs2
    mod = smf.ols(vs, df).fit()
    coef = mod.params[f'Remote:{label}']
    Coefficients.append(coef)
    Lower_Bounds.append(coef-(1.96*mod.bse[f'Remote:{label}']))
    Upper_Bounds.append(coef+(1.96*mod.bse[f'Remote:{label}']))
    Error_Bars.append(1.96*mod.bse[f'Remote:{label}'])
HTE_df = pd.DataFrame({'Market': mkts20, 'Interaction Coefficient': Coefficients, 
                       'Lower Bound': Lower_Bounds, 'Upper Bound': Upper_Bounds, 
                       'Error': Error_Bars})

# Hybrid/On-Site Matching

In [ ]:
#filtering to only on-site and remote postings with information on market, industry, and company size
df_h_os = data.copy()[(data['Remoteness']!='Remote')]
df_h_os = df_h_os[(df_h_os['Market']!='Not Available')&(df_h_os['Industry']!='Not Given')]
#treatment indicator
df_h_os['Hybrid'] = np.where(df_h_os['Remoteness']=='Hybrid', 1, 0)
#binning posting age, informed by the above visualization
df_h_os['Posting_Age_Bin'] = pd.cut(df_h_os['Days_Up'], [0, 1.0, 212.0])
df_h_os['Posting_Age_Bin'] = df_h_os['Posting_Age_Bin'].astype(str)
#binning mean salary infos -- most are 0
df_h_os['Pay_Info_Annual_Mean'] = (df_h_os['Pay_Floor_Annual']+df_h_os['Pay_Ceiling_Annual'])/2
df_h_os['Pay_Info_Mean_Category'] = (pd.cut(df_h_os['Pay_Info_Annual_Mean'], 
                                               bins=[i*10000 for i in range(26)], 
                                               include_lowest=True)).astype(str)
df_h_os['Pay_Info_Mean_Category'] = np.where(df_h_os['Pay_Info_Annual_Mean']>250000, 'Over $250,000', df_h_os['Pay_Info_Mean_Category'])
df_h_os['Pay_Info_Mean_Category'] = np.where(df_h_os['Pay_Ceiling_Annual']==0, 0, df_h_os['Pay_Info_Mean_Category'])
#binning years experience requirement info
df_h_os['yrs_exp_bin'] = pd.cut(df_h_os['Required_Years_Experience'], 
                                   bins=[i*3 for i in range(4)], 
                                   include_lowest=True)
df_h_os['yrs_exp_bin'] = np.where(df_h_os['Required_Years_Experience']>=10, '10 or more', df_h_os['yrs_exp_bin'])
#creating a list of 6 matching variables
matching_variables2 = ['Market', 'Industry', 'Company_Size', 'Pay_Info_Mean_Category', 'yrs_exp_bin', 'Posting_Age_Bin']
#grouping by that list AND treatment status, then ID'ing duplicates based on matching variables, we can 
#detect where in the covariate space potential matches are present (before gauging text similarity)
matchspots2 = (df_h_os.groupby(matching_variables2 + ['Hybrid'])['Job_Title'].
              count().reset_index()).rename(columns={'Job_Title': 'Observations'})
matchspots2 = (matchspots2[matchspots2.duplicated(subset=matching_variables2)]
              .drop(columns=['Hybrid', 'Observations']))

#Now finally I'll trawl through those covariate spaces and log t/c pairings similar enough on title as matches
matches3 = [] #to populate with matches
match_id = 0 #to keep track of what matched with what

for matchspot in range(len(matchspots2)): #i.e. for each row of promising covariate values
    #filter the data to observations in that covariate space via a one-to-many inner join
    df = matchspots2.iloc[matchspot, :].to_frame().T
    df = df_h_os.join(df.set_index(matching_variables2), 
                       on=matching_variables2, 
                       how='inner', 
                       lsuffix='_l', 
                       rsuffix='_r').reset_index(drop=True)
    #record a cutoff pt. between existing and forthcoming columns; only the latter will be used to gauge similarity
    col_count = df.shape[1]
    #a vector of titles, stripped of any references to treatment with the title_cleaner function
    titles = [title_cleaner(title) for title in df['Job_Title'].copy()]
    #initializing a model that records all unigrams as features,  fitting it on titles, and cleaning future col names
    vect = CountVectorizer(ngram_range=(1,1))
    vect.fit(titles)
    feats = [feat.replace(' ', '') for feat in vect.get_feature_names()]
    #appending each posting's new features; use of Jaccard distance to gauge similarity necessitates Boolean
    titles_matrix = vect.transform(titles)
    text_df = pd.DataFrame(titles_matrix.toarray().astype(bool), columns=feats).reset_index(drop=True)
    df = pd.concat([df, text_df], axis=1)
    #features in hand, I'll cycle through each treated unit in the covariate space and find
    #the control neighbor nearest to it based on those features (conditional on exact-matching legwork already done)
    treated_df = df.query('Hybrid==1').reset_index(drop=True)
    control_df = df.query('Hybrid==0').reset_index(drop=True)
    #the model is re-fit for every treated unit in case I want to match without replacement,
    #though the order-dependence of that might really hurt/destabilize the matching process
    for i in range(len(treated_df)):
        #a control unit model; predictions with it surface control neighbors
        #(outcome variable Apps per Day not used)
        control_neighbor_finder = (NearestNeighbors(n_neighbors=1, metric='jaccard').
                                   fit(np.array(control_df.iloc[:, col_count:]), control_df['Apps_per_Day'])
                                  )
        #the treated unit's text features as boolean values
        tdatum = np.array(treated_df.iloc[i, col_count:].to_frame().T, dtype=bool)
        #the treated unit's distance from its nearest neighbor and the NN itself's index value in control_df
        dist, neigh_loc = control_neighbor_finder.kneighbors(tdatum, n_neighbors=1)
        dist, neigh_loc = dist[0][0], neigh_loc[0][0] #extracting scalar values
        #if it's pretty close in job title to a control title in the covariate space, record both
        if dist < .6: #this is subjective. But I think strict enough using Jaccard distance
            matches3.append(treated_df.iloc[i, :col_count].to_frame().T.assign(Title_Match_Id=match_id))
            matches3.append(control_df.iloc[neigh_loc, :col_count].to_frame().T.assign(Title_Match_Id=match_id))
            match_id += 1 #increment the match identifier
        else:
            pass #since the treated unit's possible match was a poor one, discard it
#turn the list of matched observations into a dataframe
matches3 = pd.concat(matches3)
#finally, matching completed, it's maybe worth noting how small the unique set of control observations is
#if way smaller, we're relying on a relatively small set of control units, increasing variance
#a way to offset this is to increase the distance threshold (e.g. from .6 to .7), which of course will increase bias
#another way is to match without replacement, which also increases bias because this also worsens average match quality
t2 = matches3.query("Hybrid==1")
c2 = matches3.query("Hybrid==0")
print('The sample size is now', len(matches3))
print(f'However, this is composed of {len(t2)} treated units')
unique_c_count2 = c2[~c2.duplicated(subset=['Job_Title', 'Time Up', 'Date_Scraped'])].shape[0]
print(f'and {unique_c_count2} control units.')

#### Some More Processing+Estimation

In [ ]:
#joining market sizes according to Census data and handling duplicate market names
matches3 = matches3.merge(census_data, on='Market', how='left')
matches3.loc[matches3['Location'].str.contains('Portland, ME'), 'Population'] = 68313 
matches3.loc[matches3['Location'].str.contains('Louisville, CO'), 'Population'] = 20975 
matches3.loc[matches3['Location'].str.contains('Wyoming, MN'), 'Population'] = 7791
matches3.loc[matches3['Location'].str.contains('Arlington, VA'), 'Population'] = 232965
matches3.loc[matches3['Location'].str.contains('Lexington, MA'), 'Population'] = 33792
matches3.loc[matches3['Location'].str.contains('Riverside, RI'), 'Population'] = 15912
matches3.loc[matches3['Location'].str.contains('Newark, DE'), 'Population'] = 31155
matches3.loc[matches3['Location'].str.contains('Rochester, MN'), 'Population'] = 121465
matches3.loc[matches3['Location'].str.contains('Peoria, IL'), 'Population'] = 111666
matches3.loc[matches3['Location'].str.contains('Charleston, WV'), 'Population'] = 48018
matches3.loc[matches3['Location'].str.contains('Columbia, MO'), 'Population'] = 126853
matches3.loc[matches3['Location'].str.contains('Columbia, MD'), 'Population'] = 105412
matches3.loc[matches3['Location'].str.contains('Hamilton, OH'), 'Population'] = 62947
matches3.loc[matches3['Location'].str.contains('Greenville, OH'), 'Population'] = 12715
matches3.loc[matches3['Location'].str.contains('Ogden, IL'), 'Population'] = 721
matches3.loc[matches3['Location'].str.contains('Auburn, NY'), 'Population'] = 26664
matches3.loc[matches3['Location'].str.contains('Plymouth, MI'), 'Population'] = 9313
matches3.loc[matches3['Location'].str.contains('Union City, GA'), 'Population'] = 27359
matches3.loc[matches3['Location'].str.contains('Columbus, OH'), 'Population'] = 906528
matches3.loc[matches3['Location'].str.contains('Columbus, NE'), 'Population'] = 24123
matches3.loc[matches3['Location'].str.contains('Columbus, GA'), 'Population'] = 197485
matches3.loc[matches3['Location'].str.contains('Deerfield, IL'), 'Population'] = 19079
matches3.loc[matches3['Location'].str.contains('Wilton, CA'), 'Population'] = 5224
matches3.loc[matches3['Location'].str.contains('Brooklyn, NY'), 'Population'] = 2577000
matches3.loc[matches3['Location'].str.contains('Piedmont, SC'), 'Population'] = 6122
matches3['Population_hundred_thousands'] = matches3['Population']/100000
#re-casting outcome / treatment variables
matches3['Apps_per_Day_log'] = matches3['Apps_per_Day_log'].astype(float)
matches3['Hybrid'] = matches3['Hybrid'].astype(int)
matches3['Days_Up'] = matches3['Days_Up'].astype(float)
matches3_copy = matches3.copy()
matches3_copy['Software_Engineer'] = np.where(matches3_copy['Job_Title'].str.contains('Software'), 1, 0)
matches3_copy['Analyst'] = np.where(matches3_copy['Job_Title'].str.contains('Analyst'), 1, 0)
matches3_copy['Associate'] = np.where(matches3_copy['Job_Title'].str.contains('Associate'), 1, 0)
matches3_copy['Senior'] = np.where(matches3_copy['Job_Title'].str.contains('Senior'), 1, 0)
co_size_df = pd.get_dummies(matches3_copy['Company_Size']).drop(columns='10,001+ employees')
co_size_df.columns = ['UnderFiveKEmployees', 'UnderTenEmployees', 'UnderFiftyEmployees', 'UnderFiveHEmployees', 
                      'UnderTenKEmployees', 'UnderOneKEmployees', 'UnderTwoHEmployees', 'EmployeesNotGiven']
co_sizes = '+'.join(list(co_size_df.columns))
cos_todummy = list(matches3_copy['Company'].value_counts()[:20].index)
cos_df = pd.get_dummies(matches3_copy['Company'])[cos_todummy]
cos_df = cos_df.rename(columns={'Grant Thornton LLP (US)': 'GrantThorntonLLP'})
cos_df.columns = [re.sub("[,.-/'3&]", '', col).replace(' ', '') for col in cos_df.columns]
cos = '+'.join(list(cos_df.columns))
mkts_todummy = list(matches3_copy['Market'].value_counts()[:20].index)
mkts_df = pd.get_dummies(matches3_copy['Market'])[mkts_todummy]
mkts_df.columns = [re.sub("[,.-/']", '', col).replace(' ', '') for col in mkts_df.columns]
mkts = '+'.join(list(mkts_df.columns))
matches3_copy = pd.concat([matches3_copy, co_size_df, cos_df, mkts_df], axis=1)
smf.ols('Apps_per_Day_log~Hybrid*Population_hundred_thousands+Easy_Apply+Promoted', matches3_copy).fit().summary()